# Adversarial Benchmarking across models

This notebook includes the code to benchmark supervised and unsupervised ViTs against different adversarial attacks.

#### Install, Paths and Parameters

In [1]:
import os
from pathlib import Path
import getpass
import numpy as np
import time
import torch
from torch import nn
from tqdm import tqdm
import random
import sys
from torch.utils.data import DataLoader

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.train import validate_network
from src.model.data import *

# Custom imports
import torchattacks
from torchattacks import *
import torch.optim as optim
from torchvision import transforms as pth_transforms

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/cluster/scratch/thobauma/data/ori/')

ORI_PATH = Path(DATA_PATH, 'validation')
ORI_LABEL_PATH = Path(ORI_PATH,'labels.csv')
ORI_IMAGES_PATH = Path(ORI_PATH,'images')

In [2]:
INDEX_SUBSET = get_random_indexes(n_samples=3000) # Randomly sample data

BATCH_SIZE = 30
NUM_WORKERS= 0
PIN_MEMORY=True

DEVICE = 'cuda'

In [3]:
len(INDEX_SUBSET)

3000

## Load data

In [19]:
dataset = AdvTrainingImageDataset(ORI_IMAGES_PATH, ORI_LABEL_PATH, ORIGINAL_TRANSFORM, index_subset=INDEX_SUBSET, return_reduced=False)
loader = DataLoader(dataset, batch_size=50, pin_memory=True, shuffle=True)

# Evaluate DINO ViT

#### Load model

In [7]:
def get_dino_small():
    model_small, linear_classifier_small = get_dino(model_name='vit_small', patch_size=16, n_last_blocks=4, avgpool_patchtokens=False, device='cpu')
    dino_small = ViTWrapper(model_small, linear_classifier_small, device='cpu', n_last_blocks=4, avgpool_patchtokens=False)
    return dino_small

def get_dino_base():
    model_base, linear_classifier_base = get_dino(model_name='vit_base', patch_size=16, n_last_blocks=1, avgpool_patchtokens=True, device='cpu')
    dino_base = ViTWrapper(model_base, linear_classifier_base, device='cpu', n_last_blocks=1, avgpool_patchtokens=True)
    return dino_base

#### Wrap models

In [6]:
#### DINO ViT/B-16
dino_base = get_dino_base()

#### DINO ViT/S-16
dino_small = get_dino_small()

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_base built.
Embed dim 1536
We load the reference pretrained linear weights from dino_vitbase16_pretrain/dino_vitbase16_linearweights.pth.
Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights from dino_deitsmall16_pretrain/dino_deitsmall16_linearweights.pth.


#### Compute clean accuracy

In [31]:
def evaluate_dino(model_wrap, loader):
    clean_correct = 0
    total = len(loader.dataset)
    start = time.time()
    model_wrap.cuda()

    with torch.no_grad():
        for images, labels, _ in tqdm(loader):

            cuda_images = images.to(DEVICE)
            clean_outputs = model_wrap(cuda_images)
            labels = labels.to(DEVICE)

            _, pre_clean = torch.max(clean_outputs.data, 1)

            clean_correct += (pre_clean == labels).sum()

    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Clean accuracy: %.2f %%' % (100 * float(clean_correct) / total))
    model_wrap.cpu()

In [32]:
evaluate_dino(dino_small, loader)

100%|██████████| 60/60 [00:31<00:00,  1.90it/s]

Total elapsed time (sec): 31.52
Clean accuracy: 78.47 %


In [33]:
evaluate_dino(dino_base, loader)

100%|██████████| 60/60 [00:39<00:00,  1.51it/s]

Total elapsed time (sec): 39.91
Clean accuracy: 78.87 %


In [34]:
torch.cuda.empty_cache()

# Evaluate supervised ViT

#### Load model

We use pretrained model from: https://github.com/lukemelas/PyTorch-Pretrained-ViT

In [8]:
from PIL import Image
import torch
from torchvision import transforms
from pytorch_pretrained_vit import ViT

class CustomViT(ViT):
    def __init__(self):
        super().__init__('B_16_imagenet1k', pretrained=True)
        self.transform = transforms.Compose([
    transforms.Normalize(0.5, 0.5),
])
        
    def __call__(self, x):
        x = self.transform(x)
        return self.forward(x)

In [9]:
def get_supervised_vit():
    sup_vit = CustomViT().to('cpu').eval()
    return sup_vit

#### Load data

In [20]:
sup_dataset = AdvTrainingImageDataset(ORI_IMAGES_PATH, ORI_LABEL_PATH, transforms.Compose([
    transforms.Resize((384, 384)), 
    transforms.ToTensor(),
]), index_subset=INDEX_SUBSET, return_reduced=False)
sup_loader = DataLoader(sup_dataset, batch_size=10, pin_memory=True, shuffle=True)

#### Compute clean accuracy

In [9]:
def evaluate_supervised_vit(sup_vit, loader):
    sup_vit.cuda()
    clean_correct = 0
    total = len(loader.dataset)
    start = time.time()

    with torch.no_grad():
        for images, labels, _ in tqdm(loader):

            cuda_images = images.to(DEVICE)
            clean_outputs = sup_vit(cuda_images)
            labels = labels.to(DEVICE)

            _, pre_clean = torch.max(clean_outputs.data, 1)

            clean_correct += (pre_clean == labels).sum()

    sup_vit.cpu()
    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Clean accuracy: %.2f %%' % (100 * float(clean_correct) / total))

100%|██████████| 300/300 [01:30<00:00,  3.32it/s]

Total elapsed time (sec): 90.57
Clean accuracy: 84.13 %


In [ ]:
evaluate_supervised_vit(sup_vit, sup_loader)

# Evaluate ResNet-50

In [10]:
from torchvision.models.resnet import ResNet, Bottleneck

class CustomResNet(ResNet):
    def __init__(self, classifier=None):
        super(CustomResNet, self).__init__(block=Bottleneck, layers=[3, 4, 6, 3])
        self.load_state_dict(torch.load("/cluster/scratch/jrando/resnet/resnet.pth"))
        
    def _forward_impl(self, x):
        # Normalize
        transform = pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        x = transform(x)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [11]:
def get_resnet():
    return CustomResNet().cpu()

In [24]:
def evaluate_resnet(resnet, loader):
    resnet.cuda()
    clean_correct = 0
    total = len(loader.dataset)
    start = time.time()

    with torch.no_grad():
        for images, labels, _ in tqdm(loader):
            
            cuda_images = images.to(DEVICE)
            clean_outputs = resnet(cuda_images)
            labels = labels.to(DEVICE)

            _, pre_clean = torch.max(clean_outputs.data, 1)

            clean_correct += (pre_clean == labels).sum()

    resnet.cpu()
    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Clean accuracy: %.2f %%' % (100 * float(clean_correct) / total))

In [25]:
evaluate_resnet(resnet, loader)

100%|██████████| 60/60 [00:51<00:00,  1.17it/s]

Total elapsed time (sec): 51.16
Clean accuracy: 75.43 %


In [17]:
torch.cuda.empty_cache()

# Attacks
We use TorchAttack library. See: https://github.com/Harry24k/adversarial-attacks-pytorch

#### Create attacks and test on every model

In [ ]:
# delete models if created before
del resnet
del sup_vit
del dino_small
del dino_base

In [25]:
model_mapping = {'dino_base': get_dino_base,
                'dino_small': get_dino_small,
                'resnet': get_resnet,
                'sup_vit': get_supervised_vit}

In [26]:
def get_attack(model, eps=0.03, alpha=(0.03*2)/3, steps=3):
    return  PGD(model, eps=eps, alpha=alpha, steps=steps)

In [30]:
images_generated = {}

In [ ]:
for model_name in model_mapping.keys():
    img_loader = sup_loader if model_name=="sup_vit" else loader
    images_generated[model_name] = []
    model = model_mapping[model_name]().cuda()
    atk = get_attack(model)
    
    print("-"*70)
    print(model_name, atk)

    for images, labels, _ in tqdm(img_loader):

        labels = labels.to(DEVICE)
        adv_images = atk(images, labels)
        images_generated[model_name].append(adv_images.cpu())

        del images
        del labels
        del adv_images
        torch.cuda.empty_cache()
    
    del model
    del atk

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_base built.
Embed dim 1536


  0%|          | 0/60 [00:00<?, ?it/s]

We load the reference pretrained linear weights from dino_vitbase16_pretrain/dino_vitbase16_linearweights.pth.
----------------------------------------------------------------------
dino_base PGD(model_name=ViTWrapper, device=cuda:0, eps=0.03, alpha=0.02, steps=3, random_start=True, attack_mode=default, return_type=float)


  0%|          | 0/60 [00:00<?, ?it/s]

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights from dino_deitsmall16_pretrain/dino_deitsmall16_linearweights.pth.
----------------------------------------------------------------------
dino_small PGD(model_name=ViTWrapper, device=cuda:0, eps=0.03, alpha=0.02, steps=3, random_start=True, attack_mode=default, return_type=float)


 85%|████████▌ | 51/60 [01:02<00:11,  1.25s/it]